# 1. Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn.metrics
from sklearn.model_selection import GridSearchCV
from tools import utils, prop_metrics
from tools.read_data import read_data
from tools.PCA_dim_reduction import PCA_dim_reduction, binary_classification_data
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

#Classifiers
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from tools import DT_implementation, QDA_implementation

# 2. Classifiers
### This section will include all of the classifiers used in this project except DT

## 2.1 Quadratic Discriminant Analysis Classifier
### 2.1.1 Quadratic Discriminant Analysis on multiclass (6) data preprocessed by PCA dimensionality reduction (first 5 components)

In [2]:
# Get PC's from train/test data
train_x, train_y, test_x, test_y = PCA_dim_reduction('data/df_train.csv', 'data/df_test.csv', 5)

# Get class observations in arrays in a list
class_list = QDA_implementation.preprocess_QDA(train_x, train_y)

# Get class means, covariance matrices and probabilities
mean_list, covmat_list, prob_list = QDA_implementation.get_parameters_QDA(class_list, train_x)

# Get y predictions (class that maximises the discriminant function for each observation)
ypred = QDA_implementation.QDA_class_scoring(test_x, test_y, class_list, covmat_list, mean_list, prob_list)

# Get performance report on the classification metrics
prop_metrics.performance_report(test_y, ypred, len(class_list))


Confusion matrix for prediction:
 [[20. 14.  4.  0.  0.  0.]
 [ 0.  7.  0.  1.  1.  0.]
 [ 1.  1.  1.  0.  0.  0.]
 [ 0.  1.  0.  3.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  2.  8.]]


Accuracy for prediction:
 0.6


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.526 		|	 0.952 		|	 0.678

Class 2 |	 0.778 		|	 0.304 		|	 0.438

Class 3 |	 0.333 		|	 0.2 		|	 0.25

Class 5 |	 0.6 		|	 0.75 		|	 0.667

Class 6 |	 0 		|	 0.0 		|	 0

Class 7 |	 0.8 		|	 0.889 		|	 0.842


Weighted F1 score:
 0.551

Macro F1 score:
 0.479


### 2.1.2 Quadratic Discriminant Analysis on Binary classes (window/non-window) data preprocessed by  PCA dimensionality reduction (first 5 components)

In [3]:
# Get PC's from train/test BINARY data
train_x, train_y, test_x, test_y = binary_classification_data('data/df_train.csv', 'data/df_test.csv')

mask = train_y == 1
class1 = train_x[mask, :]

mask = train_y == 2
class2 = train_x[mask, :]

# Class list for combined classes
class_list = [class1, class2]

# Get class means, covariance matrices and probabilities
mean_list, covmat_list, prob_list = QDA_implementation.get_parameters_QDA(class_list, train_x)

# Get y predictions (class that maximises the discriminant function for each observation)
ypred = QDA_implementation.QDA_class_scoring(test_x, test_y, class_list, covmat_list, mean_list, prob_list)

# Get performance report on the classification metrics
prop_metrics.performance_report(test_y, ypred, len(class_list))


Confusion matrix for prediction:
 [[47.  1.]
 [ 2. 15.]]


Accuracy for prediction:
 0.9538461538461539


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.979 		|	 0.959 		|	 0.969

Class 2 |	 0.882 		|	 0.938 		|	 0.909


Weighted F1 score:
 0.954

Macro F1 score:
 0.939


## 2.2 Support Vector Machine Classifier
### 2.2.1 Support Vector Machine classifier on multiclass (6) data

In [4]:
train, test = read_data()

svm = Pipeline((("scaler", StandardScaler()),
                        ("svm_clf", SVC(kernel="rbf",
                        degree=5, C=4/3, gamma=0.1))))
                
svm.fit(train[:,:-1], train[:,-1])
y_predicted = svm.predict(test[:,:-1])
y_actual = test[:,-1]
        
prop_metrics.performance_report(y_actual, y_predicted, n_classes=6)


Confusion matrix for prediction:
 [[18.  1.  2.  0.  1.  0.]
 [ 3. 19.  3.  1.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  2.  0.  3.  0.  0.]
 [ 0.  1.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  8.]]


Accuracy for prediction:
 0.7692307692307693


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.818 		|	 0.857 		|	 0.837

Class 2 |	 0.704 		|	 0.826 		|	 0.76

Class 3 |	 0 		|	 0.0 		|	 0

Class 5 |	 0.6 		|	 0.75 		|	 0.667

Class 6 |	 0.667 		|	 0.667 		|	 0.667

Class 7 |	 1.0 		|	 0.889 		|	 0.941


Weighted F1 score:
 0.742

Macro F1 score:
 0.645


### 2.2.2 Support Vector Machine classifier on binary (window/non-window) data

In [5]:
train, test = read_data()

for idx,i in enumerate(train):
    if i[-1] == 1 or i[-1] == 2 or i[-1] == 3:
        train[idx,-1] = 1
    else:
        train[idx,-1] = 2
        
for idx,i in enumerate(test):
    if i[-1] == 1 or i[-1] == 2 or i[-1] == 3:
        test[idx,-1] = 1
    else:
        test[idx,-1] = 2
        
        
bi_svm = Pipeline((("scaler", StandardScaler()),
                        ("svm_clf", SVC(kernel="rbf",
                        degree=1, C=2/3, gamma=0.1))))
                
bi_svm.fit(train[:,:-1], train[:,-1])
bi_y_predicted = bi_svm.predict(test[:,:-1])
bi_y_actual = test[:,-1]
        
prop_metrics.performance_report(bi_y_actual, bi_y_predicted, n_classes=2)


Confusion matrix for prediction:
 [[46.  3.]
 [ 3. 13.]]


Accuracy for prediction:
 0.9076923076923077


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.939 		|	 0.939 		|	 0.939

Class 2 |	 0.812 		|	 0.812 		|	 0.812


Weighted F1 score:
 0.908

Macro F1 score:
 0.876


## 2.3 K-Nearest Neighbors classifier on 2 features (from dimentionality reduction)
### 2.3.1 K-Nearest Neighbors classifier on multiclass (6) data with top 2 linear discrimnant components

In [6]:
# Read in data and split features and labels
train, test = read_data()
train_x, train_y, test_x, test_y = train[:,:-1], train[:,-1], test[:,:-1], test[:,-1]

# Standardize data
mean,std = np.mean(train_x, axis=0), np.std(train_x, axis=0)

train_x, test_x = (train_x - mean) / std, (test_x - mean) / std

# LD transformation with eigen decomposition
lda = LinearDiscriminantAnalysis(solver='eigen')

train_x = lda.fit_transform(train_x, train_y)
test_x = lda.transform(test_x)

# Define the classifier
knn = KNeighborsClassifier(4)

# Fit with the 2 first LD components
knn.fit(train_x[:,:2],train_y)

# Predict with the 2 first LD components
y_pred = knn.predict(test_x[:,:2])

# Performance metrics
prop_metrics.performance_report(test_y, y_pred, 6)


Confusion matrix for prediction:
 [[17. 10.  2.  0.  1.  0.]
 [ 2. 10.  3.  1.  0.  0.]
 [ 2.  1.  0.  0.  0.  0.]
 [ 0.  2.  0.  3.  0.  1.]
 [ 0.  0.  0.  0.  2.  1.]
 [ 0.  0.  0.  0.  0.  7.]]


Accuracy for prediction:
 0.6


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.567 		|	 0.81 		|	 0.667

Class 2 |	 0.625 		|	 0.435 		|	 0.513

Class 3 |	 0.0 		|	 0.0 		|	 0

Class 5 |	 0.5 		|	 0.75 		|	 0.6

Class 6 |	 0.667 		|	 0.667 		|	 0.667

Class 7 |	 1.0 		|	 0.778 		|	 0.875


Weighted F1 score:
 0.586

Macro F1 score:
 0.554


### 2.3.2 K-Nearest Neighbors classifier on binary (window/non-window) data with top 2 linear discriminant components

In [7]:
# Read in data and split features and labels
train, test = read_data()
train_x, train_y, test_x, test_y = train[:,:-1], train[:,-1], test[:,:-1], test[:,-1]

# Standardize data
mean,std = np.mean(train_x, axis=0), np.std(train_x, axis=0)

train_x, test_x = (train_x - mean) / std, (test_x - mean) / std

# LD transformation with eigen decomposition
lda = LinearDiscriminantAnalysis(solver='eigen')

train_x = lda.fit_transform(train_x, train_y)
test_x = lda.transform(test_x)

# Seperate the test and train classes into window and non-window
train_y[train_y < 4] = 1
train_y[train_y > 4] = 2

test_y[test_y < 4] = 1
test_y[test_y > 4] = 2

# Define the classifier
knn = KNeighborsClassifier(1)

# Fit on the training data
knn.fit(train_x[:,:2], train_y)

# Predict labels for test data
y_pred = knn.predict(test_x[:,:2])

# Performance Metrics
prop_metrics.performance_report(test_y, y_pred, 2)


Confusion matrix for prediction:
 [[44.  2.]
 [ 5. 14.]]


Accuracy for prediction:
 0.8923076923076924


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.957 		|	 0.898 		|	 0.926

Class 2 |	 0.737 		|	 0.875 		|	 0.8


Weighted F1 score:
 0.895

Macro F1 score:
 0.863


## 2.4 Soft Voting Classifier
### 2.4.1 Soft Voting classifier on multiclass (6) data preprocessed by PCA dimensionality reduction (first  5 components)

In [9]:
# Read in the 5 component pca-data
train_x, train_y, test_x, test_y = PCA_dim_reduction('data/df_train.csv', 'data/df_test.csv', 5)

# Define the base models and hyper parameters
base_models = [('QDA', QuadraticDiscriminantAnalysis()), 
               ('DT', DecisionTreeClassifier(max_depth=2, min_samples_split=9)), 
               ('SVM', SVC(C=1000, gamma=0.01, probability=True)), 
               ('KNN', KNeighborsClassifier(n_neighbors=2))]

# Define the meta model
meta_model = VotingClassifier(estimators=base_models, voting = 'soft', weights=[3, 3, 5, 3]) # Weight chosen by finetuning

# Fit the meta model to the train data
meta_model.fit(train_x, train_y)

# Committee's prediction on the train data
pred_y = meta_model.predict(test_x)

# Performance of the ensemble of models' voting
prop_metrics.performance_report(test_y, meta_model.predict(test_x), 6)


Confusion matrix for prediction:
 [[21.  1.  4.  0.  1.  0.]
 [ 0. 22.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  3.  0.  1.]
 [ 0.  0.  0.  0.  2.  0.]
 [ 0.  0.  0.  0.  0.  8.]]


Accuracy for prediction:
 0.8615384615384616


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.778 		|	 1.0 		|	 0.875

Class 2 |	 0.917 		|	 0.957 		|	 0.936

Class 3 |	 0 		|	 0.0 		|	 0

Class 5 |	 0.75 		|	 0.75 		|	 0.75

Class 6 |	 1.0 		|	 0.667 		|	 0.8

Class 7 |	 1.0 		|	 0.889 		|	 0.941


Weighted F1 score:
 0.827

Macro F1 score:
 0.717


### 2.4.2 Soft Voting classifier on Binary classes (window/non-window) data preprocessed by  PCA dimensionality reduction (first 5 components)

In [11]:
# Read in binary window/non-window 5 component pca-data
train_x, train_y, test_x, test_y = binary_classification_data('data/df_train.csv', 'data/df_test.csv')

# Try model from search above, if none then use earlier model
base_models = [('QDA', QuadraticDiscriminantAnalysis()), 
               ('DT', DecisionTreeClassifier(max_depth=2, min_samples_split=8)), 
               ('SVM', SVC(C=100, gamma=0.1, probability=True)), 
               ('KNN', KNeighborsClassifier(n_neighbors=1))]

meta_model = VotingClassifier(estimators= base_models, voting='soft')

# fit the model
meta_model.fit(train_x, train_y)

# get predictions from the voting classifier
pred_y = meta_model.predict(test_x)

# print performance metrics from the predictions on the test data
prop_metrics.performance_report(test_y, meta_model.predict(test_x), 2)


Confusion matrix for prediction:
 [[47.  1.]
 [ 2. 15.]]


Accuracy for prediction:
 0.9538461538461539


Metrics for classes
_______________________________________________________________________________
Class	|	Precision	|	Recall		|	F1 Score
_______________________________________________________________________________

Class 1 |	 0.979 		|	 0.959 		|	 0.969

Class 2 |	 0.882 		|	 0.938 		|	 0.909


Weighted F1 score:
 0.954

Macro F1 score:
 0.939
